# 硬件
:label:`sec_hardware`

构建高性能系统不仅需要对算法和模型有很好的理解以捕捉问题的统计方面，同时至少也需要对底层硬件有一定的了解。当前章节不能替代正式的硬件和系统设计课程，但它可能作为理解为什么某些算法比其他算法更高效以及如何实现良好的吞吐量的起点。一个好的设计可以轻松地使性能提高一个数量级，反过来，这可以在能够训练网络（例如，在一周内）和完全无法训练之间做出区别（在3个月内，从而错过截止日期）。我们将从计算机开始。然后我们会更仔细地查看CPU和GPU。最后我们再放大视角，回顾多台计算机是如何在服务器中心或云端连接在一起的。

![每个程序员都应该知道的延迟数据。](../img/latencynumbers.png)
:label:`fig_latencynumbers`

不耐烦的读者也许可以通过:numref:`fig_latencynumbers`来满足需求。它取自Colin Scott的[互动帖子](https://people.eecs.berkeley.edu/%7Ercs/research/interactive_latency.html)，该帖子很好地概述了过去十年的进步。原始数据来自Jeff Dean的[2010年斯坦福演讲](https://static.googleusercontent.com/media/research.google.com/en//people/jeff/Stanford-DL-Nov-2010.pdf)。
下面的讨论解释了这些数字的一些理由以及它们如何指导我们设计算法。下面的讨论非常高级且简略。很明显，*这不是*一门正规课程的替代品，而是旨在为统计建模者提供足够的信息以做出合适的设计决策。对于计算机架构的深入概述，我们建议读者参考:cite:`Hennessy.Patterson.2011`或相关主题的最新课程，如[Arste Asanovic](http://inst.eecs.berkeley.edu/%7Ecs152/sp19/)开设的课程。

## 计算机

大多数深度学习研究者和实践者都能接触到一台拥有大量内存、计算能力及某种形式加速器（如GPU）或多块加速器的计算机。一台计算机由以下关键组件组成：

* 处理器（也称为CPU），能够执行我们给它的程序（除了运行操作系统和其他许多事情），通常由8个或更多核心组成。
* 内存（RAM）用于存储和检索计算结果，如权重向量和激活值，以及训练数据。
* 以太网网络连接（有时多个），速度范围从1 GB/s到100 GB/s。高端服务器上可以找到更先进的互连。
* 高速扩展总线（PCIe）将系统连接到一个或多个GPU。服务器最多可配备8个加速器，通常采用高级拓扑结构连接，而台式机系统根据用户的预算和电源供应大小配备1或2个。
* 耐用存储，如磁性硬盘驱动器、固态驱动器，很多情况下通过PCIe总线连接。它提供了训练数据到系统的高效传输，并根据需要存储中间检查点。

![计算机组件的连接。](../img/mobo-symbol.svg)
:label:`fig_mobo-symbol`

如:numref:`fig_mobo-symbol`所示，大多数组件（网络、GPU和存储）都通过PCIe总线与CPU相连。它由直接连接到CPU的多条通道组成。例如AMD的Threadripper 3有64条PCIe 4.0通道，每条通道的数据传输能力达到16 Gbit/s。内存直接连接到CPU，总带宽高达100 GB/s。

当我们在计算机上运行代码时，我们需要将数据移动到处理器（CPU或GPU），执行计算，然后将结果移出处理器回到RAM和持久存储中。因此，为了获得良好的性能，我们需要确保这一切无缝工作，没有一个系统成为主要瓶颈。例如，如果我们不能快速加载图像，处理器将没有任何工作可做。同样，如果我们不能快速将矩阵移动到CPU（或GPU），其处理单元将会饿死。最后，如果我们希望跨网络同步多台计算机，后者不应拖慢计算速度。一种选择是交错通信和计算。让我们更详细地看一下各个组件。


## 内存

最基本的记忆体是用来存储需要随时访问的数据。目前，CPU RAM通常是[DDR4](https://en.wikipedia.org/wiki/DDR4_SDRAM)类型的，每模块提供20-25 GB/s的带宽。每个模块有一个64位宽的总线。通常成对使用内存模块以允许多个通道。CPU有2到4个内存通道，即它们具有40GB/s到100 GB/s的峰值内存带宽。通常每个通道有两个库。例如AMD的Zen 3 Threadripper有8个插槽。

虽然这些数字令人印象深刻，但它们只讲述了部分故事。当我们想要从内存中读取一部分内容时，我们首先需要告诉内存模块信息的位置。也就是说，我们首先需要将*地址*发送到RAM。一旦完成这一点，我们可以选择只读取单个64位记录或长序列记录。后者称为*突发读取*。简而言之，向内存发送地址并设置传输大约需要100 ns（具体取决于使用的内存芯片的具体时序系数），每次后续传输仅需0.2 ns。简言之，第一次读取的成本是后续读取的500倍！注意，我们每秒最多可以执行10,000,000次随机读取。这表明我们应该尽可能避免随机内存访问，而是使用突发读取（和写入）。

当我们考虑到我们有多个*库*时，情况会更加复杂。每个库可以相对独立地读取内存。这意味着两件事。
一方面，如果随机读取均匀分布在内存中，那么有效的随机读取次数最多可以提高4倍。这也意味着进行随机读取仍然是一个坏主意，因为突发读取也是4倍快。另一方面，由于内存对齐到64位边界，将任何数据结构与相同的边界对齐是一个好主意。当设置了适当的标志时，编译器几乎[自动](https://en.wikipedia.org/wiki/Data_structure_alignment)这样做。好奇的读者被鼓励复习关于DRAM的讲座，如Zeshan Chishti的[讲座](http://web.cecs.pdx.edu/%7Ezeshan/ece585_lec5.pdf)。

GPU内存面临更高的带宽要求，因为它们比CPU有更多的处理单元。大体上有两种方法来解决这个问题。第一种是使内存总线显著变宽。例如，NVIDIA的RTX 2080 Ti有一个352位宽的总线。这允许在同一时间传输更多的信息。第二，GPU使用特定的高性能内存。消费级设备，如NVIDIA的RTX和Titan系列通常使用[GDDR6](https://en.wikipedia.org/wiki/GDDR6_SDRAM)芯片，聚合带宽超过500 GB/s。另一种选择是使用HBM（高带宽内存）模块。它们使用非常不同的接口，并通过专用硅晶圆直接与GPU连接。这使得它们非常昂贵，通常仅限于高端服务器芯片，如NVIDIA Volta V100系列加速器。毫不奇怪，由于前者的成本较高，GPU内存通常*远小于*CPU内存。就我们的目的而言，大体上它们的性能特性相似，只是快得多。我们可以忽略本书的目的细节。只有在调整GPU内核以实现高吞吐量时，它们才重要。

## 存储

我们看到RAM的一些关键特性是*带宽*和*延迟*。对于存储设备来说也是如此，只是差异可能更大。

### 硬盘驱动器

*硬盘驱动器* (HDDs) 已经使用了半个多世纪。简单地说，它们包含多个旋转盘片，带有可以定位在任意轨道上读取或写入的磁头。高端磁盘在9个盘片上最多可以存储16 TB。HDD的一个主要优点是它们相对便宜。它们的许多缺点之一是典型的灾难性故障模式和相对较高的读取延迟。

要理解后者，考虑HDD以大约7,200 RPM（每分钟转数）的速度旋转。如果它们更快，就会因离心力作用在盘片上而破裂。这在访问磁盘上的特定扇区时带来了一个重大缺点：我们需要等待盘片旋转到位（我们可以移动磁头，但不能加速实际的磁盘）。因此，可能需要超过8毫秒才能获取请求的数据。一个常见的表达方式是说HDD可以以大约100 IOPs（每秒输入/输出操作）的速度运行。这个数字在过去二十年里基本保持不变。更糟糕的是，增加带宽也同样困难（它的速度在100-200 MB/s左右）。毕竟，每个磁头读取一条比特轨道，因此比特率仅随着信息密度的平方根而缩放。因此，HDD正迅速退化为归档存储和非常大型数据集的低级别存储。


### 固态硬盘

固态硬盘 (SSD) 使用闪存来持久存储信息。这允许*更快*地访问存储的记录。现代SSD可以以100,000至500,000 IOPs的速度运行，即比HDD快3个数量级。此外，它们的带宽可以达到1-3GB/s，即比HDD快一个数量级。这些改进听起来几乎是天方夜谭。确实，由于SSD的设计方式，它们附带了以下注意事项。

* SSD以块（256 KB或更大）的形式存储信息。它们只能整体写入，这需要相当长的时间。因此，SSD上的按位随机写入性能非常差。同样，写入数据通常需要很长时间，因为必须先读取块，擦除后再用新信息重写。如今，SSD控制器和固件已经开发出了算法来缓解这种情况。尽管如此，写入仍然可能慢得多，特别是对于QLC（四层单元）SSD。提高性能的关键是维护一个*队列*的操作，优先读取并在可能的情况下以大块写入。
* SSD中的存储单元磨损相对较快（往往在几千次写入后就已经发生）。磨损均衡保护算法能够将降级分散到许多单元上。尽管如此，不建议将SSD用于交换文件或大型日志文件聚合。
* 最后，带宽的巨大提升迫使计算机设计师将SSD直接连接到PCIe总线上。能够处理这种操作的驱动器被称为NVMe（非易失性内存增强型），可以使用多达4条PCIe通道。这相当于在PCIe 4.0上最高可达8GB/s。

### 云存储

云存储提供了可配置的性能范围。也就是说，存储分配给虚拟机是动态的，既包括数量也包括速度，由用户选择。我们建议用户在延迟过高时（例如，在训练许多小记录时）增加配置的IOPs数量。

## CPU

中央处理单元 (CPU) 是任何计算机的核心部件。它们由几个关键组件组成：能够执行机器代码的*处理器核心*、连接它们的*总线*（具体拓扑结构在不同处理器型号、代际和供应商之间差异显著），以及*缓存*，以实现比从主内存读取更高的带宽和更低的延迟。最后，几乎所有现代CPU都包含*矢量处理单元*，以帮助实现高性能线性代数和卷积，这是媒体处理和机器学习中常见的操作。

![Intel Skylake消费者级四核CPU。](../img/skylake.svg)
:label:`fig_skylake`

:numref:`fig_skylake` 描述了一款Intel Skylake消费者级四核CPU。它集成了GPU、缓存和连接四个核心的环形总线。外围设备，如以太网、WiFi、蓝牙、SSD控制器和USB，要么是芯片组的一部分，要么直接通过PCIe连接到CPU。

### 微架构

每个处理器核心由一组相当复杂的组件组成。尽管不同代际和供应商之间的细节有所不同，但基本功能基本上是标准的。前端加载指令并尝试预测将采取的路径（例如，控制流）。然后将指令从汇编代码解码为微指令。汇编代码往往不是处理器执行的最低级别代码。相反，复杂的指令可能被解码为一组较低级别的操作。这些操作随后由实际的执行核心处理。通常，后者能够同时执行许多操作。例如，:numref:`fig_cortexa77` 中的ARM Cortex A77核心能够同时执行多达8个操作。

![ARM Cortex A77微架构。](../img/a77.svg)
:label:`fig_cortexa77`

这意味着高效的程序可能能够在每个时钟周期执行多于一条指令，前提是它们可以独立执行。并非所有单元都是平等创建的。有些专精于整数指令，而另一些则优化了浮点性能。为了增加吞吐量，处理器也可能同时遵循分支指令中的多个代码路径，然后丢弃未采用的分支的结果。这就是为什么分支预测单元很重要（在前端），以便只追求最有前途的路径。

### 向量化

深度学习极其依赖计算。因此，为了让CPU适用于机器学习，需要在一个时钟周期内执行许多操作。这是通过矢量单元实现的。它们有不同的名称：在ARM上称为NEON，在x86上（最近一代）称为[AVX2](https://en.wikipedia.org/wiki/Advanced_Vector_Extensions)单元。共同之处在于它们能够执行SIMD（单指令多数据）操作。:numref:`fig_neon128` 显示了如何在一个时钟周期内在ARM上添加8个短整数。

![128位NEON向量化。](../img/neon128.svg)
:label:`fig_neon128`

根据架构选择，这样的寄存器最长可达512位，允许组合多达64对数字。例如，我们可能在乘以两个数并将结果加到第三个数上，这也被称为融合乘加。Intel的[OpenVino](https://01.org/openvinotoolkit)利用这些实现了在服务器级CPU上进行深度学习的可观吞吐量。不过，请注意，这个数字完全被GPU所能实现的吞吐量所掩盖。例如，NVIDIA的RTX 2080 Ti拥有4,352个CUDA核心，每个核心都能够随时处理此类操作。

### 缓存

考虑以下情况：我们有一个如:numref:`fig_skylake` 上所示的适度的4核CPU，运行频率为2 GHz。
此外，假设我们的IPC（每时钟指令数）计数为1，并且单元启用了256位宽度的AVX2。进一步假设用于AVX2操作的至少一个寄存器需要从内存中检索。这意味着CPU每时钟周期消耗$4 \times 256 \textrm{ bit} = 128 \textrm{ bytes}$的数据。除非我们能够每秒向处理器传输$2 \times 10^9 \times 128 = 256 \times 10^9$字节的数据，否则处理单元将会饥饿。不幸的是，这种芯片的内存接口仅支持20-40 GB/s的数据传输，即少了一个数量级。解决办法是尽可能避免从内存中加载*新的*数据，而是将其本地缓存在CPU上。这就是缓存派上用场的地方。通常使用以下名称或概念：

* **寄存器**严格来说并不属于缓存的一部分。它们有助于暂存指令。也就是说，CPU寄存器是CPU可以以时钟速度无延迟访问的内存位置。CPU有数十个寄存器。编译器（或程序员）应有效使用寄存器。例如，C编程语言有一个`register`关键字。
* **L1缓存**是抵御高内存带宽需求的第一道防线。L1缓存很小（典型大小可能是32-64 KB），并且经常分为数据和指令缓存。当数据在L1缓存中找到时，访问速度非常快。如果在那里找不到，则搜索会沿着缓存层次结构向下进行。
* **L2缓存**是下一站。根据架构设计和处理器大小，它们可能是独占的。它们可能只由给定的核心访问，或者由多个核心共享。L2缓存比L1更大（通常每个核心256-512 KB），但速度较慢。此外，要访问L2中的内容，我们首先需要检查以确认数据不在L1中，这增加了少量额外的延迟。
* **L3缓存**由多个核心共享，可以相当大。AMD的Epyc 3服务器CPU在其多个芯片上分布了高达256 MB的缓存。更典型的数字在4-8 MB范围内。

预测接下来需要哪些内存元素是芯片设计中的一个关键优化参数。例如，建议按*顺序*遍历内存，因为大多数缓存算法都会尝试*预读*而不是反向读取。同样，保持内存访问模式局部化是提高性能的好方法。

添加缓存是一把双刃剑。一方面，它们确保处理器核心不会因数据不足而饥饿。同时，它们增加了芯片尺寸，占用了本可用于增加处理能力的空间。此外，*缓存未命中*可能会很昂贵。考虑最坏的情况，*伪共享*，如:numref:`fig_falsesharing`所示。当处理器1上的线程请求数据时，内存位置已在处理器0上缓存。为了获取它，处理器0需要停止正在做的事情，将信息写回主内存，然后让处理器1从内存中读取。在此操作期间，两个处理器都在等待。这样的代码在多处理器上运行时*可能比*高效的单处理器实现*更慢*。这也是缓存大小有实际限制的原因之一（除了物理尺寸之外）。

![伪共享（图片来自Intel）。](../img/falsesharing.svg)
:label:`fig_falsesharing`

## GPU和其他加速器

可以说，如果没有GPU，深度学习就不会成功。同理，也可以合理地认为，由于深度学习的发展，GPU制造商的财富大幅增加。这种硬件和算法的共同进化导致了这样一个局面：无论好坏，深度学习已成为首选的统计建模范式。因此，了解GPU及相关加速器（如TPU :cite:`Jouppi.Young.Patil.ea.2017`）的具体优势是有益的。

值得注意的是，实践中常常做出的一种区分：加速器要么针对训练进行优化，要么针对推理进行优化。对于后者，我们只需要在网络中计算前向传播。不需要为反向传播存储任何中间数据。此外，我们可能不需要非常精确的计算（FP16或INT8通常足够）。另一方面，在训练过程中，所有中间结果都需要存储以计算梯度。此外，累积梯度需要更高的精度以避免数值下溢（或上溢）。这意味着FP16（或混合精度FP32）是最小要求。所有这些都需要更快和更大的内存（HBM2 vs. GDDR6）以及更多的处理能力。例如，NVIDIA的[Turing](https://devblogs.nvidia.com/nvidia-turing-architecture-in-depth/) T4 GPU针对推理进行了优化，而V100 GPU更适合训练。

回想:numref:`fig_neon128`中的向量化。在处理器核心中添加向量单元允许我们显著提高吞吐量。例如，在:numref:`fig_neon128`中的示例中，我们能够同时执行16个操作。
首先，
如果我们添加不仅优化向量之间的操作，还优化矩阵之间的操作呢？这种策略导致了张量核心（将在稍后介绍）的产生。
其次，如果我们添加更多的核心呢？简而言之，这两个策略总结了GPU的设计决策。:numref:`fig_turing_processing_block`给出了一个基本处理块的概览。它包含16个整数和16个浮点单元。此外，两个张量核心加速了与深度学习相关的狭窄子集的附加操作。每个流式多处理器由四个这样的块组成。

![NVIDIA Turing处理块（图片由NVIDIA提供）。](../img/turing-processing-block.png)
:width:`150px`
:label:`fig_turing_processing_block`

接下来，12个流式多处理器被分组为图形处理集群，构成了高端TU102处理器。充足的内存通道和L2缓存补充了这一设置。:numref:`fig_turing`包含了相关细节。设计这种设备的一个原因是，可以根据需要添加或删除单个块，以允许更紧凑的芯片，并处理产量问题（有缺陷的模块可能不会被激活）。幸运的是，编写这类设备的程序对于普通的深度学习研究人员来说隐藏得很好，被CUDA和框架代码层层包裹。特别地，只要资源可用，GPU上可以同时执行多个程序。尽管如此，了解设备的局限性还是很有必要的，以免选择不适合设备内存的模型。

![NVIDIA Turing架构（图片由NVIDIA提供）](../img/turing.png)
:width:`350px`
:label:`fig_turing`

值得详细提及的最后一个方面是*张量核心*。它们是近期趋势的一个例子，即添加专门针对深度学习优化的电路。例如，TPU添加了用于快速矩阵乘法的脉动阵列 :cite:`Kung.1988`。在那种设计中，目的是支持非常少（第一代TPU只有一个）的大规模运算。张量核心则位于另一端。它们针对涉及$4 \times 4$到$16 \times 16$矩阵的小规模运算进行了优化，具体取决于它们的数值精度。:numref:`fig_tensorcore`概述了这些优化。

![NVIDIA Turing中的张量核心（图片由NVIDIA提供）。](../img/tensorcore.jpg)
:width:`400px`
:label:`fig_tensorcore`

显然，当我们优化计算时，最终会做出某些妥协。其中之一是GPU不太擅长处理中断和稀疏数据。虽然有一些例外，如[Gunrock](https://github.com/gunrock/gunrock) :cite:`Wang.Davidson.Pan.ea.2016`，但稀疏矩阵和向量的访问模式与GPU擅长的高带宽突发读取操作不匹配。匹配这两个目标是当前的研究热点。参见例如[DGL](http://dgl.ai)，这是一个针对图上深度学习优化的库。


## 网络和总线

每当单个设备不足以进行优化时，我们需要将数据传输到设备并从中同步处理。这时网络和总线就派上了用场。我们有几个设计参数：带宽、成本、距离和灵活性。
在一端，我们有Wi-Fi，它的覆盖范围相当好，非常易于使用（毕竟无需电线），价格低廉，但提供的带宽和延迟相对平庸。任何一个理智的机器学习研究人员都不会用它来构建服务器集群。在下文中，我们将重点关注适合深度学习的互连技术。

* **PCIe**是一种用于非常高带宽点对点连接的专用总线（在PCIe 4.0的16通道插槽中，每通道最高可达32 GB/s）。延迟在个位数微秒（5 μs）的数量级。PCIe链接是宝贵的。处理器只有有限数量的PCIe通道：AMD的EPYC 3有128条通道，Intel的Xeon最多有48条通道；在桌面级CPU上，Ryzen 9有20条通道，Core i9有16条通道。由于GPU通常有16条通道，这限制了可以以全带宽连接到CPU的GPU数量。毕竟，它们需要与其他高带宽外设（如存储和以太网）共享链路。就像RAM访问一样，大批量传输是优选的，因为这样可以减少包开销。
* **以太网**是最常用的连接计算机的方式。虽然它比PCIe慢得多，但它非常便宜且安装方便，覆盖的距离也更长。低端服务器的典型带宽为1 GBit/s。高端设备（例如，云中的[C5实例](https://aws.amazon.com/ec2/instance-types/c5/)）提供10到100 GBit/s的带宽。与之前的所有情况一样，数据传输具有显著的开销。请注意，我们几乎从未直接使用原始以太网，而是在物理互连之上运行协议（如UDP或TCP/IP）。这会增加进一步的开销。像PCIe一样，以太网旨在连接两个设备，例如，一台计算机和一个交换机。
* **交换机**允许我们以一种方式连接多个设备，使得其中任意一对设备可以同时进行（通常是全带宽）点对点连接。例如，以太网交换机可能以高交叉带宽连接40台服务器。请注意，交换机并不是传统计算机网络独有的。即使PCIe通道也可以[切换](https://www.broadcom.com/products/pcie-switches-bridges/pcie-switches)。例如，当将大量GPU连接到主机处理器时就是这种情况，如[P2实例](https://aws.amazon.com/ec2/instance-types/p2/)。
* **NVLink**是PCIe的一种替代方案，用于非常高的带宽互连。它每条链路提供高达300 Gbit/s的数据传输速率。服务器GPU（Volta V100）有六条链路，而消费级GPU（RTX 2080 Ti）只有一条链路，以降低的100 Gbit/s速率运行。我们建议使用[NCCL](https://github.com/NVIDIA/nccl)来实现在GPU之间的高速数据传输。



## 更多延迟数据

:numref:`table_latency_numbers` 和 :numref:`table_latency_numbers_tesla` 的摘要来自[Eliot Eshelman](https://gist.github.com/eshelman)，他维护了一个更新版本的数字作为[GitHub gist](https://gist.github.com/eshelman/343a1c46cb3fba142c1afdcdeec17646)。

:常见延迟数据。

| 操作 | 时间 | 备注 |
| :----------------------------------------- | -----: | :---------------------------------------------- |
| L1缓存引用/命中                     | 1.5 ns | 4个周期                                        |
| 浮点加/乘/FMA                | 1.5 ns | 4个周期                                        |
| L2缓存引用/命中                     |   5 ns | 12 ~ 17个周期                                  |
| 分支预测失败                          |   6 ns | 15 ~ 20个周期                                  |
| L3缓存命中（未共享缓存）              |  16 ns | 42个周期                                       |
| L3缓存命中（在另一个核心中共享）      |  25 ns | 65个周期                                       |
| Mutex锁/解锁                          |  25 ns |                                                 |
| L3缓存命中（在另一个核心中修改）    |  29 ns | 75个周期                                       |
| L3缓存命中（在远程CPU插座上）        |  40 ns | 100 ~ 300个周期（40 ~ 116 ns）                  |
| QPI跳到另一个CPU（每次跳）         |  40 ns |                                                 |
| 64MB内存引用（本地CPU）          |  46 ns | 在Broadwell E5-2690v4上的TinyMemBench             |
| 64MB内存引用（远程CPU）         |  70 ns | 在Broadwell E5-2690v4上的TinyMemBench             |
| 256MB内存引用（本地CPU）         |  75 ns | 在Broadwell E5-2690v4上的TinyMemBench             |
| Intel Optane随机写                  |  94 ns | UCSD非易失性系统实验室                   |
| 256MB内存引用（远程CPU）        | 120 ns | 在Broadwell E5-2690v4上的TinyMemBench             |
| Intel Optane随机读                   | 305 ns | UCSD非易失性系统实验室                   |
| 通过100 Gbps HPC结构发送4KB          |   1 μs | MVAPICH2 over Intel Omni-Path                   |
| 使用Google Snappy压缩1KB            |   3 μs |                                                 |
| 通过10 Gbps以太网发送4KB             |  10 μs |                                                 |
| 随机向NVMe SSD写入4KB             |  30 μs | DC P3608 NVMe SSD（QOS 99%为500μs）            |
| 通过NVLink GPU传输1MB到/从           |  30 μs | NVIDIA 40GB NVLink上的~33GB/s                 |
| 通过PCI-E GPU传输1MB到/从           |  80 μs | PCIe 3.0 x16链路上的~12GB/s                  |
| 从NVMe SSD随机读取4KB            | 120 μs | DC P3608 NVMe SSD（QOS 99%）                     |
| 从NVMe SSD连续读取1MB        | 208 μs | DC P3608 NVMe SSD上的~4.8GB/s                    |
| 随机向SATA SSD写入4KB             | 500 μs | DC S3510 SATA SSD（QOS 99.9%）                   |
| 从SATA SSD随机读取4KB            | 500 μs | DC S3510 SATA SSD（QOS 99.9%）                   |
| 在同一数据中心内的往返               | 500 μs | 单向ping约为250μs                          |
| 从SATA SSD连续读取1MB        |   2 ms | DC S3510 SATA SSD上的~550MB/s                    |
| 从磁盘连续读取1MB            |   5 ms | 服务器HDD上的~200MB/s                           |
| 随机磁盘访问（寻道+旋转）         |  10 ms |                                                 |
| 从加州到荷兰再到加州发送数据包       | 150 ms |                                                 |
:label:`table_latency_numbers`

:NVIDIA Tesla GPU的延迟数据。

| 操作 | 时间 | 备注 |
| :------------------------------ | -----: | :---------------------------------------- |
| GPU共享内存访问        |  30 ns | 30~90个周期（银行冲突会增加延迟） |
| GPU全局内存访问        | 200 ns | 200~800个周期                            |
| 在GPU上启动CUDA内核       |  10 μs | 主CPU指示GPU启动内核    |
| 通过NVLink GPU传输1MB到/从 |  30 μs | NVIDIA 40GB NVLink上的~33GB/s           |
| 通过PCI-E GPU传输1MB到/从  |  80 μs | PCI-Express x16链路上的~12GB/s         |
:label:`table_latency_numbers_tesla`

## 总结

* 设备对操作都有开销。因此，重要的是尽量减少多次小批量传输，而改为少数几次大批量传输。这适用于RAM、SSD、网络和GPU。
* 向量化是性能的关键。确保您了解您的加速器的具体能力。例如，一些Intel Xeon CPU在INT8操作方面特别出色，NVIDIA Volta GPU在FP16矩阵-矩阵操作方面表现出色，而NVIDIA Turing在FP16、INT8和INT4操作方面表现出色。
* 训练期间（以及在较小程度上推断期间）由于数据类型过小而导致的数值溢出可能是一个问题。
* 别名可以显著降低性能。例如，在64位CPU上，内存对齐应该基于64位边界。在GPU上，保持卷积大小对齐是个好主意，比如对齐到张量核心。
* 匹配您的算法和硬件（例如，内存占用和带宽）。当参数能够放入缓存时，可以获得极大的速度提升（数量级）。
* 我们建议您在验证实验结果之前，在纸上草拟出一种新算法的性能。数量级或以上的差异是值得关注的理由。
* 使用分析器调试性能瓶颈。
* 训练和推断硬件在价格和性能方面有不同的最佳点。

## 练习

1. 编写C代码测试相对于外部内存接口对齐或错位访问内存是否有速度上的差异。提示：小心缓存效应。
1. 测试顺序访问内存与按给定步幅访问内存之间的速度差异。
1. 如何测量CPU上的缓存大小？
1. 如何布局多个内存通道以实现最大带宽？如果您有许多小线程，又该如何布局？
1. 企业级HDD以10,000 rpm的速度旋转。在最坏情况下，HDD需要花费的绝对最小时间是多少才能读取数据（您可以假设磁头几乎瞬间移动）？为什么2.5英寸HDD在商业服务器中变得流行（相对于3.5英寸和5.25英寸驱动器）？
1. 假设HDD制造商将存储密度从每平方英寸1 Tbit提高到5 Tbit。您可以在2.5英寸HDD的环上存储多少信息？内外圈是否有差异？
1. 从8位数据类型到16位数据类型，所需的硅面积大约增加了四倍。为什么？为什么NVIDIA会在其Turing GPU中添加INT4操作？
1. 从前往后读取内存与从后往前读取相比有多快？这个数字在不同的计算机和CPU供应商之间是否有所不同？为什么？编写C代码并进行实验。
1. 您能测量磁盘的缓存大小吗？典型HDD的缓存有多大？SSD需要缓存吗？
1. 测量通过以太网发送消息的包开销。查找UDP和TCP/IP连接之间的差异。
1. 直接内存访问允许除CPU以外的设备直接读写内存。为什么这是一个好主意？
1. 查看Turing T4 GPU的性能数据。为什么从FP16到INT8和INT4的性能“仅”翻倍？
1. 从旧金山到阿姆斯特丹再返回的最短时间应该是多少？提示：您可以假设距离为10,000公里。


[讨论](https://discuss.d2l.ai/t/363)